# seeks to locate in classified entities within the given corpus. it can be locationsn names anything. etcc

There are some pre defined entities in the model actually, but there will be a situation where we needs to train our own entity asper the domain.

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.pipe_names

['tagger', 'parser', 'ner']

tagger parser and named entity recogniser

# lets look at the example

In [ ]:
doc = nlp("Australia wants to force Facebook and Google to pay media companies for news")

In [ ]:
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

Australia 0 9 GPE
Facebook and Google 25 44 ORG


It is ok now, but what if we want some of our custom named entities to be also identified as entities especially in financial and banking domains.

In [ ]:
doc = nlp("I do not have money to pay my credit card account")

In [ ]:
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

# no entity in this statement 

In [ ]:
doc = nlp("what is the process to open a new savings account")

In [ ]:
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

again no entity in this text as well.

here saving account and credit card is a product the bank have and payment and open is an activity.

So, need to train for this custome elements

so, i am going to train a model for all this to be sorted out.

In [ ]:
train = [
         ("Money transfer from my checking account is not working", {"entities": [(6, 13, "Activity"), (23, 39, 'Product')]}),
         (" I want to check balance in my savings account", {"entities": [(16, 23, "Activity"), (30, 45, 'Product')]}),
         ("I suspect a fraud in my credit card account", {"entities": [(12, 17, "Activity"), (24, 35, 'Product')]}),
         ("I am here for opening a new savings account", {"entities": [(14, 21, "Activity"), (28, 43, 'Product')]}),
         ("Your credit card is in past due status", {"entities": [(23, 31, "Activity"), (5, 16, 'Product')]})
]

In [ ]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [ ]:
ner = nlp.get_pipe("ner")

In [ ]:
for _,annotations in train:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

now we need t add these custome made to the ner 

In [ ]:
# disabling all the pipelines, if the pipeline not equal to ner,assign it to the disable pipes

disable_pipes = [pipe for pipe in nlp.pipe_names if pipe!='ner']

In [ ]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path

with nlp.disable_pipes(*disable_pipes):
  optimizer = nlp.resume_training()

  for iteration in range(100):

    random.shuffle(train)
    losses = {}

    batches = minibatch(train, size=compounding(1.0, 4.0, 1.001))
    for batch in batches:
      text, annotation = zip(*batch)
      nlp.update(
          text, annotation, drop = 0.5, losses = losses, sgd = optimizer 
      )
      print("Losses", losses)

Losses {'ner': 7.82286264589134}
Losses {'ner': 18.09132142951799}
Losses {'ner': 23.713039688515124}
Losses {'ner': 32.10974466203608}
Losses {'ner': 39.18176854546159}
Losses {'ner': 7.403922836488846}
Losses {'ner': 14.53615510176894}
Losses {'ner': 23.311481552328544}
Losses {'ner': 31.93050105323978}
Losses {'ner': 39.60926009920554}
Losses {'ner': 10.238402935621707}
Losses {'ner': 19.09171195293044}
Losses {'ner': 28.96287579499969}
Losses {'ner': 36.42088160173271}
Losses {'ner': 47.99666831151817}
Losses {'ner': 5.0047439646123895}
Losses {'ner': 13.946703018637814}
Losses {'ner': 21.608711870248648}
Losses {'ner': 26.41083100524869}
Losses {'ner': 33.859423639604614}
Losses {'ner': 10.28347950170064}
Losses {'ner': 16.615366558097094}
Losses {'ner': 24.22988012677365}
Losses {'ner': 30.728826582080956}
Losses {'ner': 42.88865846360409}
Losses {'ner': 8.599374008830637}
Losses {'ner': 14.169407587483875}
Losses {'ner': 23.535164995319747}
Losses {'ner': 33.20888229293337}
Loss

In [ ]:
for text, _ in train:
  doc = nlp(text)
  print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []
Entities []
Entities []
Entities []
Entities [('credit card', 'Product'), ('past due', 'Activity')]


feeding more data would definitely do.

In [ ]:
from spacy import displacy

doc = nlp("what is the process to open a new credit card")

for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style = 'ent', jupyter = True)


credit card 34 45 Product


In [ ]:
doc = nlp("My credit card payment will be past due status")
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

credit card 3 14 Product


It is doing some decent job with the very less data we feeded in.

In [ ]:
doc = nlp("Australia wants to force Facebook and Google to pay media companies for news")
for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

here comes the check, now it is not recgnisizing the facebbok and google as organizations, this is due to the forgetting of the model. rfere the site " https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting "

in tis case what we need to do is we need to take the existing data and train it along with the new data. It is a known issue in spacy and other nlp packages.

also, we can have two models:
one will have empty model with our text 
and the second will be with existing models and
finally combine them and show
